<a href="https://colab.research.google.com/github/arthurcornelio88/stable-audio-tools-sam/blob/main/scrapping/scrapping-analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
!git clone https://github.com/arthurcornelio88/stable-audio-tools-sam.git


Cloning into 'stable-audio-tools-sam'...
remote: Enumerating objects: 1492, done.
remote: Counting objects: 100% (576/576), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 1492 (delta 500), reused 488 (delta 484), pack-reused 916
Receiving objects: 100% (1492/1492), 68.89 MiB | 24.58 MiB/s, done.
Resolving deltas: 100% (995/995), done.


In [105]:
import pandas as pd
import numpy as np

# Read the Excel file
url = 'https://github.com/arthurcornelio88/stable-audio-tools-sam/raw/main/scrapping/Export%2010k.xlsx'
df = pd.read_excel(url)
df.head()

,Title,Link to unique page,Image,artist_url,label,Length,Keywords
0,Dancing under the stars. Background music for ...,https://pixabay.com/music/beats-dancing-under-...,https://cdn.pixabay.com/audio/2024/07/17/16-48...,https://pixabay.com/users/white_records-32584949/,White_Records,0:25,background
1,In Slow Motion (Inspiring Ambient Lounge),https://pixabay.com/music/future-bass-in-slow-...,https://cdn.pixabay.com/audio/2024/06/25/10-06...,https://pixabay.com/users/soundbay-601514/,soundbay,1:58,future bass\nabstract
2,Dancing under the stars. Background music for ...,https://pixabay.com/music/beats-dancing-under-...,https://cdn.pixabay.com/audio/2024/07/17/16-48...,https://pixabay.com/users/white_records-32584949/,White_Records,1:06,background
3,Dancing under the stars. Background music for ...,https://pixabay.com/music/beats-dancing-under-...,https://cdn.pixabay.com/audio/2024/07/17/16-48...,https://pixabay.com/users/white_records-32584949/,White_Records,0:48,background
4,Dancing under the stars. background music for ...,https://pixabay.com/music/beats-dancing-under-...,https://cdn.pixabay.com/audio/2024/07/17/16-26...,https://pixabay.com/users/white_records-32584949/,White_Records,2:12,background


In [80]:
# Convert Length column to numeric format after removing non-numeric characters and colons
df['Length'] = df['Length'].astype(str).str.replace(r'[^\d:]', '', regex=True)
df['Length'] = pd.to_numeric(df['Length'].astype(str).str.replace(':', '', regex=False), errors='coerce')

# Impute missing values in `Length` with the mean
mean_length = df['Length'].mean()
df['Length'] = df['Length'].fillna(mean_length)

# Print the updated DataFrame
print(df.head())

                                               Title  \
0  Dancing under the stars. Background music for ...   
1          In Slow Motion (Inspiring Ambient Lounge)   
2  Dancing under the stars. Background music for ...   
3  Dancing under the stars. Background music for ...   
4  Dancing under the stars. background music for ...   

                                 Link to unique page  \
0  https://pixabay.com/music/beats-dancing-under-...   
1  https://pixabay.com/music/future-bass-in-slow-...   
2  https://pixabay.com/music/beats-dancing-under-...   
3  https://pixabay.com/music/beats-dancing-under-...   
4  https://pixabay.com/music/beats-dancing-under-...   

                                               Image  \
0  https://cdn.pixabay.com/audio/2024/07/17/16-48...   
1  https://cdn.pixabay.com/audio/2024/06/25/10-06...   
2  https://cdn.pixabay.com/audio/2024/07/17/16-48...   
3  https://cdn.pixabay.com/audio/2024/07/17/16-48...   
4  https://cdn.pixabay.com/audio/2024/07/17/16

In [81]:
len(df[df['Length']>600]) / len(df) # 7% of the music have more than 10 minutes. Filter it ?

0.07180261832829808

In [82]:
# Filter the DataFrame to keep only rows where the `Length` value does not match the pattern "\d+:\d+" or "\d+"
problematic_rows = df[~df['Length'].astype(str).str.replace(r'[^\d:]', '', regex=True).str.match(r'^\d+:\d+$|^\d+$')]

# If there are any rows matching the filter criteria, print these rows and their count
if not problematic_rows.empty:
    print(f"\n{len(problematic_rows)} problematic rows with invalid Length values:")
    print(problematic_rows.to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("\nNo problematic rows found.")



No problematic rows found.


In [83]:
# Print the column names and their data types
print("Column names and datatypes:")
print(df.info())

Column names and datatypes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9930 entries, 0 to 9929
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Title                9930 non-null   object 
 1   Link to unique page  9930 non-null   object 
 2   Image                8234 non-null   object 
 3   artist_url           9808 non-null   object 
 4   label                9808 non-null   object 
 5   Length               9930 non-null   float64
 6   Keywords             9930 non-null   object 
dtypes: float64(1), object(6)
memory usage: 543.2+ KB
None


In [85]:
# function to replace separator of keywords from '\n' to '\ '.
df['Keywords'] = df['Keywords'].astype(str).str.replace(r'\n,?', ', ', regex=True)

In [87]:
df[['Keywords']]

,Keywords
0,background
1,"future bass, abstract"
2,background
3,background
4,background
...,...
9925,"phonk, electronic, trap"
9926,"action, exciting, bass"
9927,"music, melody, guitar"
9928,"cinema, fantasy, mystical"


In [88]:
# Calculate and print the number of null values in each column
print("\nNull values per column:")
print(df.isnull().sum().to_markdown(numalign="left", stralign="left"))


Null values per column:
|                     | 0    |
|:--------------------|:-----|
| Title               | 0    |
| Link to unique page | 0    |
| Image               | 1696 |
| artist_url          | 122  |
| label               | 122  |
| Length              | 0    |
| Keywords            | 0    |


In [89]:
# Calculate and print the number of duplicated rows
print(f"\nNumber of duplicated rows: {df.duplicated().sum()}")


Number of duplicated rows: 0


In [90]:
# Calculate and print the number of unique values in each column
print("\nUnique values per column:")
print(df.nunique().to_markdown(numalign="left", stralign="left"))


Unique values per column:
|                     | 0    |
|:--------------------|:-----|
| Title               | 9647 |
| Link to unique page | 9929 |
| Image               | 8233 |
| artist_url          | 818  |
| label               | 818  |
| Length              | 811  |
| Keywords            | 5565 |


In [91]:
# Calculate and print the mean and standard deviation of the `Length` column
mean_length = df['Length'].mean()
std_length = df['Length'].std()
print(f"\nMean Length: {mean_length:.2f} seconds")
print(f"Standard Deviation of Length: {std_length:.2f} seconds")


Mean Length: 304.92 seconds
Standard Deviation of Length: 439.37 seconds


In [92]:
# Identify outliers in the `Length` column (using Z-score method)
z_scores = (df['Length'] - mean_length) / std_length
outliers = df[(z_scores > 3) | (z_scores < -3)]

# Print the number of outliers found
if not outliers.empty:
    print(f"\n{len(outliers)} outliers found in Length (using Z-score method):")
    print(outliers.to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("\nNo outliers found in Length (using Z-score method).")


369 outliers found in Length (using Z-score method):
| Title                                                                                           | Link to unique page                                                                                                                              | Image                                                              | artist_url                                                | label                 | Length   | Keywords                                 |
|:------------------------------------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------------------------------------|:----------------------------------------------------------|:----------------------|:---------|:-----------------------------------------|
| Relaxing Music Vol.1                

In [94]:
'''
  We can see, then, that 3% of the music are outliers, so, before, we drop them, we need to understand their nature.
'''

len(outliers) / len(df)

0.03716012084592145

In [102]:
'''
  We will count the most frequent genres in the outliers dataframe. Instrumental, meditation and ambient surrounding
    are musics with long duration.
  Maybe we put them in another dataset ?
'''

def get_top_n_words(df, column_name, n=5):
    """
    Counts the frequency of words in a specified column of a DataFrame and returns the top n words.

    Args:
        df: The DataFrame containing the text data.
        column_name: The name of the column to analyze.
        n: The number of top words to return (default is 5).

    Returns:
        A Series containing the top n words and their frequencies.
    """
    all_words = df[column_name].str.split(', ').explode()
    word_counts = all_words.value_counts()
    return word_counts.head(n)

top_words = get_top_n_words(outliers, 'Keywords', 5)
top_words

Keywords
music                 77
instrumental music    71
meditation            57
ambient               34
surroundings          30
Name: count, dtype: int64

In [103]:
'''
  In the same time, we have a lot of advertising, background, ambient, etc that are ok in terms of lenght.
  But we will drop the outliers, because of their lenght finally (not because of their genre).
  df = df[~df['Length'].isin(outliers['Length'])]
'''

ll_words = get_top_n_words(df, 'Keywords', 5)
ll_words.head(10)

Keywords
advertising    944
background     862
music          610
ambient        543
action         462
Name: count, dtype: int64

In [101]:
# dropping outliers
new_df = df[~df['Length'].isin(outliers['Length'])]
len(new_df) / len(df)

0.9628398791540785

In [126]:
# saving new DF

df.to_csv('/content/stable-audio-tools-sam/scrapping/cleaned-export-10k.csv', index=False)
df.to_excel('/content/stable-audio-tools-sam/scrapping/cleaned_filtered-export-10k.xlsx', index=False)

In [127]:
from google.colab import userdata
import os

# Set your GitHub Personal Access Token as an environment variable
os.environ['GITHUB_TOKEN'] = userdata.get('sam_token')

%%shell

# Navigate to the repository directory
cd /content/stable-audio-tools-sam

# Stage the files
git add /content/stable-audio-tools-sam/scrapping/cleaned-export-10k.csv \
        /content/stable-audio-tools-sam/scrapping/cleaned_filtered-export-10k.xlsx

# Commit the changes with a message
git commit -m "Add cleaned data in CSV and Excel formats to scrapping folder"

# Push the changes to your GitHub repository
git push https://${GITHUB_TOKEN}@github.com/arthurcornelio88/stable-audio-tools-sam.git main



[main b7b3eaf] Add cleaned data in CSV and Excel formats to scrapping folder
 1 file changed, 0 insertions(+), 0 deletions(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 821 bytes | 821.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/arthurcornelio88/stable-audio-tools-sam.git
   00c98df..b7b3eaf  main -> main
